In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [7]:
# Load CSV file generated from WeatherPy Folder
weather_data_to_load = "output_data/weather.csv"
weather_data = pd.read_csv(weather_data_to_load)
dropna_weather_data = weather_data.dropna()
del dropna_weather_data["Unnamed: 0"]
dropna_weather_data.head(20)

,City,Cloudiness,Country,Humidity (%),Lat,Lng,Max Temp,Wind Speed,Date
0,beloha,0,MG,78,-25.1667,45.0500,67.82,10.40,1619233924
1,adrar,1,MR,15,20.5022,-10.0711,74.70,8.95,1619233904
2,asosa,35,ET,73,10.0667,34.5333,65.14,5.06,1619234077
3,urumqi,75,CN,64,43.8010,87.6005,35.60,8.95,1619233867
4,alta floresta,75,NO,64,69.9689,23.2717,32.00,8.05,1619234077
5,verkhnevilyuysk,100,RU,91,63.4453,120.3167,32.92,5.86,1619234078
6,castro,8,BR,88,-24.7911,-50.0119,52.90,3.22,1619234002
7,laguna,1,US,62,38.4210,-121.4238,61.00,10.36,1619234012
8,mys shmidta,60,RU,75,62.0589,48.6201,44.49,5.73,1619234078
9,kapaa,40,US,69,22.0752,-159.3190,77.00,18.41,1619233629


In [8]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Locations
locations = dropna_weather_data[["Lat", "Lng"]]

humidity = dropna_weather_data["Humidity (%)"].astype(float)

In [9]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=2)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [14]:
# Filter vacation with zero cloudiness
vacation_no_cloud = dropna_weather_data[dropna_weather_data["Cloudiness"] == 0]
# Filter vacation with max temp above 70 degrees F
vacation_above_70_degrees = vacation_no_cloud[vacation_no_cloud["Max Temp"] > 70]
# Filter vacation with max temp below 80 degrees F
vacation_below_80_degrees = vacation_above_70_degrees[vacation_above_70_degrees["Max Temp"] < 80]
# Filter vacation with wind speed below 10 mph
vacation_slow_wind = vacation_below_80_degrees[vacation_below_80_degrees["Wind Speed"] < 10]
# Filter vacation with humidity below 60 %
perfect_vacation = vacation_slow_wind[vacation_slow_wind["Humidity (%)"] < 60]
# Set Index
hotel_df = perfect_vacation.reset_index()
del hotel_df["index"]
hotel_df

,City,Cloudiness,Country,Humidity (%),Lat,Lng,Max Temp,Wind Speed,Date
0,kohlu,0,PK,29,29.8962,69.2520,70.27,5.17,1619234192
1,padampur,0,IN,32,29.6833,73.6167,78.40,7.09,1619234211
2,pahrump,0,US,14,36.2083,-115.9839,73.00,1.01,1619234371
3,hanumangarh,0,IN,33,29.5833,74.3167,77.67,6.96,1619234376
4,tianpeng,0,CN,53,30.9866,103.9393,73.40,8.95,1619234406
5,kijang,0,KR,43,35.2442,129.2139,71.60,5.75,1619234427
6,seoul,0,KR,26,37.5683,126.9778,71.60,8.05,1619234194
7,riyadh,0,SA,18,24.6877,46.7219,78.80,5.75,1619234394


In [15]:
vaca_locations = hotel_df[["Lat", "Lng"]]

vaca_humidity = hotel_df["Humidity (%)"].astype(float)

# Plot Heatmap
vaca_fig = gmaps.figure()

# Create heat layer
vaca_heat_layer = gmaps.heatmap_layer(vaca_locations, weights=vaca_humidity, 
                                 dissipating=False, max_intensity=50,
                                 point_radius=2.5)

# Add layer
vaca_fig.add_layer(vaca_heat_layer)

# Display figure
vaca_fig

Figure(layout=FigureLayout(height='420px'))

In [16]:
# Hotel variable
hotels = []

# Loop through narrowed down dataframe to get nearest hotel
for city in range(len(hotel_df["City"])):

    lat = hotel_df.loc[city]["Lat"]
    lng = hotel_df.loc[city]["Lng"]

    city_coords = f"{lat},{lng}"

    params = {
        "location": city_coords, 
        "types": "lodging",
        "radius": 5000,
        "key": g_key
    }

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"   

    hotel_request = requests.get(base_url, params=params)
    hotel_response = hotel_request.json()

    try:
        hotels.append(hotel_response["results"][0]["name"])
    except:
        hotels.append("Nearest hotel not found")

# Dataframe with nearest hotel
hotel_df["Hotel Name"] = hotels
hotel_df

,City,Cloudiness,Country,Humidity (%),Lat,Lng,Max Temp,Wind Speed,Date,Hotel Name
0,kohlu,0,PK,29,29.8962,69.2520,70.27,5.17,1619234192,B &R Rest House
1,padampur,0,IN,32,29.6833,73.6167,78.40,7.09,1619234211,Pensia Sadan
2,pahrump,0,US,14,36.2083,-115.9839,73.00,1.01,1619234371,Best Western Pahrump Oasis
3,hanumangarh,0,IN,33,29.5833,74.3167,77.67,6.96,1619234376,SPOT ON 66636 Cps Resorts
4,tianpeng,0,CN,53,30.9866,103.9393,73.40,8.95,1619234406,Shuangliu Love Home Business Hotel
5,kijang,0,KR,43,35.2442,129.2139,71.60,5.75,1619234427,Samyang Beach Villa
6,seoul,0,KR,26,37.5683,126.9778,71.60,8.05,1619234194,Lotte Hotel Seoul
7,riyadh,0,SA,18,24.6877,46.7219,78.80,5.75,1619234394,"InterContinental Riyadh, an IHG Hotel"


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]